<a href="https://colab.research.google.com/github/hopperrr/building-footprint-segmentation/blob/main/Building_footprints.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Source: https://github.com/santhi-2020/building-footprint-segmentation/blob/main/temp.py

In [ ]:
!pip install building-footprint-segmentation
!pip install geospatial
#!pip install rasterio
#!pip install geopandas
#!pip install leafmap
#!pip install xarray_leaflet

In [1]:
#@title Create goog.xml file for pulling satellite image
%%writefile goog.xml
<GDAL_WMS>

<!-- Data is subject to term of use detailed at http://code.google.com/intl/nl/apis/maps/terms.html and
     http://www.google.com/intl/en_ALL/help/terms_maps.html -->

    <Service name="TMS">
        <!-- <ServerUrl>http://mt.google.com/vt/lyrs=m&amp;x=${x}&amp;y=${y}&amp;z=${z}</ServerUrl> --> <!-- Map -->
        <ServerUrl>http://mt.google.com/vt/lyrs=s&amp;x=${x}&amp;y=${y}&amp;z=${z}</ServerUrl> <!-- Satellite -->
        <!-- <ServerUrl>http://mt.google.com/vt/lyrs=y&amp;x=${x}&amp;y=${y}&amp;z=${z}</ServerUrl> --> <!-- Hybrid -->
        <!-- <ServerUrl>http://mt.google.com/vt/lyrs=t&amp;x=${x}&amp;y=${y}&amp;z=${z}</ServerUrl> --> <!-- Terrain -->
        <!-- <ServerUrl>http://mt.google.com/vt/lyrs=p&amp;x=${x}&amp;y=${y}&amp;z=${z}</ServerUrl> --> <!-- Terrain, Streets and Water  -->
    </Service>
    <DataWindow>
        <UpperLeftX>-20037508.34</UpperLeftX>
        <UpperLeftY>20037508.34</UpperLeftY>
        <LowerRightX>20037508.34</LowerRightX>
        <LowerRightY>-20037508.34</LowerRightY>
        <TileLevel>20</TileLevel>
        <TileCountX>1</TileCountX>
        <TileCountY>1</TileCountY>
        <YOrigin>top</YOrigin>
    </DataWindow>
    <Projection>EPSG:3857</Projection>
    <BlockSizeX>256</BlockSizeX>
    <BlockSizeY>256</BlockSizeY>
    <BandsCount>3</BandsCount>
    <MaxConnections>5</MaxConnections>
    <Cache />
</GDAL_WMS>

Writing goog.xml


In [2]:
from osgeo import gdal
#Sample area: 1599243.53,5064967.27
#1599837.10,5065633.91
#LW: -13146232.8,3998522.4

#Paste the values into the coords array below
coords = [1599243.53,5064967.27]
ulx = coords[0]
uly = coords[1]
llx = ulx + 1000
lly = uly - 1000
ds = gdal.Open('goog.xml')
ds = gdal.Translate('/content/satellite.png', ds, format = 'PNG', projWin = [ulx, uly, llx, lly], width = 3200, height = 3200)
ds = None

In [ ]:
import cv2
import torch
import numpy as np
from building_footprint_segmentation.seg.binary.models import ReFineNet
from building_footprint_segmentation.helpers.normalizer import min_max_image_net
from building_footprint_segmentation.utils.py_network import (
    to_input_image_tensor,
    add_extra_dimension,
    convert_tensor_to_numpy,
    load_parallel_model
)
from building_footprint_segmentation.utils.operations import handle_image_size
from torch.utils import model_zoo
#Size needs to be divisible by 32 (I think). CUDA has problems with memory when image is 4000 x 4000 +
MAX_SIZE = 3200
#MAX_SIZE = 256
MODEL_URL = "https://github.com/fuzailpalnak/building-footprint-segmentation/releases/download/alpha/refine.zip"
blank_image = np.zeros((MAX_SIZE,MAX_SIZE,3), np.uint8)

def get_model():
    refine_net = ReFineNet()
    state_dict = model_zoo.load_url(MODEL_URL, progress=True, map_location="cpu")
    refine_net.load_state_dict(state_dict)
    return refine_net


def extract(original_image, model):

    original_height, original_width = original_image.shape[:2]

    if (original_height, original_width) != (MAX_SIZE, MAX_SIZE):
        original_image = handle_image_size(original_image, (MAX_SIZE, MAX_SIZE))

    # Apply Normalization
    normalized_image = min_max_image_net(img=original_image)

    tensor_image = add_extra_dimension(to_input_image_tensor(normalized_image))

    with torch.no_grad():
        # Perform prediction
        prediction = model(tensor_image)
        prediction = prediction.sigmoid()
    
    prediction_binary = convert_tensor_to_numpy(prediction[0]).reshape(
        (MAX_SIZE, MAX_SIZE)
    
    )


    prediction_3_channels = cv2.cvtColor(prediction_binary, cv2.COLOR_GRAY2RGB)
    
    print (prediction_3_channels[0])
        
    dst = cv2.addWeighted(
        blank_image,
        1,
        #(prediction_3_channels * (255,69,0)).astype(np.uint8),
        (prediction_3_channels * (255,255,255)).astype(np.uint8),
        #0.4,
        #0
        1.0,
        0,
    )
   # cv2.imshow ("test", prediction_3_channels)
   # cv2.waitKey(0)
    
    cv2.imwrite ("/content/predict.png", dst)

    return prediction_binary, prediction_3_channels, dst


def run(image_path):
    original_image = cv2.imread(image_path)
    original_image = cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB)

    model = get_model()
    # PARALLELIZE the model if gpu available
    model = load_parallel_model(model)
    
    prediction_binary, prediction_3_channels, dst = extract(original_image, model)
    return prediction_3_channels
    
   
dst = run ('/content/satellite.png')
!cp /content/satellite.png.aux.xml /content/predict.png.aux.xml

In [ ]:
from PIL import Image
import geopandas
import fiona
from google.colab import files
img = Image.open('/content/predict.png')
rgba = img.convert("RGBA")
datas = rgba.getdata()
  
newData = []
for item in datas:
    if item[0] == 255 and item[1] == 255 and item[2] == 255:  # finding black colour by its RGB value
        # storing a transparent value when we find a black colour
        newData.append((255, 255, 255, 0))
    else:
        newData.append(item)  # other colours remain unchanged
  
rgba.putdata(newData)

rgba.save("/content/rectified_transparent.png", "PNG")
!cp /content/satellite.png.aux.xml rectified_transparent.png.aux.xml

!gdal_polygonize.py "/content/rectified_transparent.png" -mask /content/rectified_transparent.png -b 1 -f "GPKG" OUTPUT.gpkg OUTPUT DN
unsimplified_gdf = geopandas.read_file("/content/OUTPUT.gpkg", layer='output')
gdf_simplified = unsimplified_gdf
gdf_simplified['geometry'] = gdf_simplified['geometry'].simplify(0.5).buffer(0)
gdf254 = gdf_simplified[(gdf_simplified['DN'] > 253)]
gdf254_4326  = gdf254.to_crs({'init': 'epsg:4326'})

In [ ]:
import leafmap
m = leafmap.Map(google_map="HYBRID", center=[14.3, 41.4], zoom=6)
#datasets need to be in EPSG:4326???
m.add_gdf(gdf254_4326, layer_name="buildings")
m

# **Bringing in code from another project to see if the polygons can be cleaned up.**

In [ ]:
!git clone https://github.com/hopperrr/RS-building-regularization.git
%cd RS-building-regularization/

In [ ]:
from PIL import Image
  
img = Image.open('/content/predict.png')
rgba = img.convert("RGBA")
datas = rgba.getdata()
  
newData = []
for item in datas:
    if item[0] >= 0 and item[1] >= 0 and item[2] >= 0:  # finding black colour by its RGB value
        # storing a transparent value when we find a black colour
        newData.append((255, 255, 255, 0))
    else:
        newData.append(item)  # other colours remain unchanged
  
rgba.putdata(newData)
rgba.save("/content/transparent_image.png", "PNG")

In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
from rdp_alg import rdp
from cal_dist_ang import cal_ang, cal_dist, azimuthAngle
from rotate_ang import Nrotation_angle_get_coor_coordinates, Srotation_angle_get_coor_coordinates
from line_intersection import line, intersection, par_line_dist, point_in_line

def boundary_regularization(img, epsilon=6):
    h, w = img.shape[0:2]

    contours, hierarchy = cv2.findContours(img, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    contours = np.squeeze(contours[0])



    contours = rdp(contours, epsilon=epsilon)

    contours[:, 1] = h - contours[:, 1]


    dists = []
    azis = []
    azis_index = []


    for i in range(contours.shape[0]):
        cur_index = i
        next_index = i+1 if i < contours.shape[0]-1 else 0
        prev_index = i-1
        cur_point = contours[cur_index]
        nest_point = contours[next_index]
        prev_point = contours[prev_index]

        dist = cal_dist(cur_point, nest_point)
        azi = azimuthAngle(cur_point, nest_point)

        dists.append(dist)
        azis.append(azi)
        azis_index.append([cur_index, next_index])



    longest_edge_idex = np.argmax(dists)
    main_direction = azis[longest_edge_idex]


    correct_points = []
    para_vetr_idxs = [] 
    for i, (azi, (point_0_index, point_1_index)) in enumerate(zip(azis, azis_index)):

        if i == longest_edge_idex:
            correct_points.append([contours[point_0_index], contours[point_1_index]])
            para_vetr_idxs.append(0)
        else:
            rotate_ang = main_direction - azi

            if np.abs(rotate_ang) < 180/4:
                rotate_ang = rotate_ang
                para_vetr_idxs.append(0)
            elif np.abs(rotate_ang) >= 90-180/4:
                rotate_ang = rotate_ang + 90
                para_vetr_idxs.append(1)


            point_0 = contours[point_0_index]
            point_1 = contours[point_1_index]
            point_middle = (point_0 + point_1) / 2

            if rotate_ang > 0:
                rotate_point_0 = Srotation_angle_get_coor_coordinates(point_0, point_middle, np.abs(rotate_ang))
                rotate_point_1 = Srotation_angle_get_coor_coordinates(point_1, point_middle, np.abs(rotate_ang))
            elif rotate_ang < 0:
                rotate_point_0 = Nrotation_angle_get_coor_coordinates(point_0, point_middle, np.abs(rotate_ang))
                rotate_point_1 = Nrotation_angle_get_coor_coordinates(point_1, point_middle, np.abs(rotate_ang))
            else:
                rotate_point_0 = point_0
                rotate_point_1 = point_1
            correct_points.append([rotate_point_0, rotate_point_1])

    correct_points = np.array(correct_points)


    final_points = []
    final_points.append(correct_points[0][0])
    for i in range(correct_points.shape[0]-1):
        cur_index = i
        next_index = i + 1 if i < correct_points.shape[0] - 1 else 0

        cur_edge_point_0 = correct_points[cur_index][0]
        cur_edge_point_1 = correct_points[cur_index][1]
        next_edge_point_0 = correct_points[next_index][0]
        next_edge_point_1 = correct_points[next_index][1]

        cur_para_vetr_idx = para_vetr_idxs[cur_index]
        next_para_vetr_idx = para_vetr_idxs[next_index]

        if cur_para_vetr_idx != next_para_vetr_idx:
            L1 = line(cur_edge_point_0, cur_edge_point_1)
            L2 = line(next_edge_point_0, next_edge_point_1)

            point_intersection = intersection(L1, L2)
            final_points.append(point_intersection)

        elif cur_para_vetr_idx == next_para_vetr_idx:
            L1 = line(cur_edge_point_0, cur_edge_point_1)
            L2 = line(next_edge_point_0, next_edge_point_1)
            marg = par_line_dist(L1, L2)

            if marg < 3:

                point_move = point_in_line(next_edge_point_0[0], next_edge_point_0[1], cur_edge_point_0[0], cur_edge_point_0[1], cur_edge_point_1[0], cur_edge_point_1[1])
                final_points.append(point_move)

                correct_points[next_index][0] = point_move
                correct_points[next_index][1] = point_in_line(next_edge_point_1[0], next_edge_point_1[1], cur_edge_point_0[0], cur_edge_point_0[1], cur_edge_point_1[0], cur_edge_point_1[1])


            else:

                add_mid_point = (cur_edge_point_1 + next_edge_point_0) / 2
                add_point_1 = point_in_line(add_mid_point[0], add_mid_point[1], cur_edge_point_0[0], cur_edge_point_0[1], cur_edge_point_1[0], cur_edge_point_1[1])
                add_point_2 = point_in_line(add_mid_point[0], add_mid_point[1], next_edge_point_0[0], next_edge_point_0[1], next_edge_point_1[0], next_edge_point_1[1])
                final_points.append(add_point_1)
                final_points.append(add_point_2)


    final_points.append(final_points[0])
    final_points = np.array(final_points)

    final_points[:, 1] = h - final_points[:, 1]
    return final_points


#ori_img1 = cv2.imread('./input_data/ori.jpg')
trans_img = cv2.imread('/content/transparent_image.png')
ori_img1 = cv2.imread('/content/predict.png')
ori_img = cv2.medianBlur(ori_img1, 5)
ori_img = cv2.cvtColor(ori_img, cv2.COLOR_BGR2GRAY)
ret, ori_img = cv2.threshold(ori_img, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)

num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(ori_img, connectivity=8)
print(num_labels)


for i in range(1, num_labels):
    img = np.zeros_like(labels)
    index = np.where(labels==i)
    img[index] = 255
    img = np.array(img, dtype=np.uint8)

    regularization_contour = boundary_regularization(img).astype(np.int32)

    #cv2.polylines(img=ori_img1, pts=[regularization_contour], isClosed=True, color=(255, 0, 0), thickness=5)
    #cv2.polylines(img=trans_img, pts=[regularization_contour], isClosed=True, color=(255, 0, 0), thickness=1)
    cv2.fillPoly(img=trans_img, pts=[regularization_contour], color =(255,0,0))

    single_out = np.zeros_like(ori_img1)
    #cv2.polylines(img=single_out, pts=[regularization_contour], isClosed=True, color=(255, 0, 0), thickness=5)
    cv2.fillPoly(img=single_out, pts = [regularization_contour], color =(255,0,0))
    cv2.imwrite('/content/RS-building-regularization/output_data/single_out_{}.png'.format(i), single_out)

cv2.imwrite('all_out.png', ori_img1)
cv2.imwrite('rectified.png', trans_img)
!cp /content/satellite.png.aux.xml rectified.png.aux.xml

In [ ]:
from PIL import Image
import geopandas
import fiona
from google.colab import files
#img = Image.open('/content/RS-building-regularization/rectified.png')
img = Image.open('/content/predict.png')
rgba = img.convert("RGBA")
datas = rgba.getdata()
  
newData = []
for item in datas:
    if item[0] == 255 and item[1] == 255 and item[2] == 255:  # finding black colour by its RGB value
        # storing a transparent value when we find a black colour
        newData.append((255, 255, 255, 0))
    else:
        newData.append(item)  # other colours remain unchanged
  
rgba.putdata(newData)
rgba.save("/content/RS-building-regularization/rectified_transparent.png", "PNG")
!cp /content/satellite.png.aux.xml rectified_transparent.png.aux.xml

!gdal_polygonize.py "/content/RS-building-regularization/rectified_transparent.png" -mask /content/RS-building-regularization/rectified.png -b 1 -f "GPKG" OUTPUT.gpkg OUTPUT DN
unsimplified_gdf = geopandas.read_file("/content/RS-building-regularization/OUTPUT.gpkg", layer='output')
gdf_simplified = unsimplified_gdf
gdf_simplified['geometry'] = gdf_simplified['geometry'].simplify(0.5).buffer(0)
gdf_simplified.to_file("simplified.gpkg", layer='simple', driver="GPKG")
#!ogr2ogr -t_srs EPSG:4326 -f GPKG out4326.gpkg simplified.gpkg
#!zip out.zip /content/RS-building-regularization/rectified_transparent.png  simplified.gpkg /content/RS-building-regularization/rectified_transparent.png.aux.xml
#!zip out.zip /content/rectified_transparent.png  out4326.gpkg /content/rectified_transparent.png.aux.xml
#files.download('out.zip')